In [1]:
# 분별 날짜 파일에 있는 날짜 기준으로 DB에서 데이터 받아와서 저장하기

import MySQLdb
import os
import numpy as np
import csv
import datetime

# MySQL DB 연결
db = MySQLdb.connect('210.102.142.13',"root", "witlab8*", "cas_db")
c = db.cursor()

# 분별 날짜
date = np.genfromtxt('../data/date_winter.csv', delimiter=',', dtype='str')
# date = np.genfromtxt('../data/test_date_summer.csv', delimiter=',', dtype='str')

# csv 파일로 내보내기
w = open('../data/db_connect_data_am.csv', 'w', encoding='utf-8')
# w = open('../data/db_connect_test_data_am.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

# csv 파일로 내보내기
w2 = open('../data/db_connect_data_pm.csv', 'w', encoding='utf-8')
# w2 = open('../data/db_connect_test_data_pm.csv', 'w', encoding='utf-8')
wr2 = csv.writer(w2)

# 각 날짜별 데이터들 DB에서 가져오고 csv 파일로 저장
# swr, mwr, lwr data 받아옴
for j in range(len(date)):
    sql = "select time(date),cas_swr, cas_mwr, cas_lwr from natural_tracker left outer join cas_wave_ratio using(date) where date(date) = '"+ str(date[j][0]) + "' order by time(date)"
    c.execute(sql)
    rows = c.fetchall()
    
     # 일출 후 6시간과 일몰 후 6시간 데이터 사용, 일출 혹은 일몰 당시 데이터가 없을 경우 가장 가까운 다른 데이터로 변환
    sunrise = datetime.datetime.strptime(date[j][1], '%H:%M:%S')
    sunset = datetime.datetime.strptime(date[j][2], '%H:%M:%S')
    
    standard = datetime.datetime.strptime('1900-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
    
    sunrise = sunrise - standard
    sunset = sunset - standard
    
    start = 0
    end = 0
    
    for i in range(len(rows)):
        if(rows[i][0] >= sunrise):
            start = i
            print(date[j][0], rows[start][0], sunrise)
            break;
    
    for i in range(len(rows)):
        if(rows[i][0] >= sunset):
            end = i
            print(date[j][0], rows[end][0], sunset)
            break;

    # start에 저장된 index부터 772개 데이터 가져와서 저장
    for l in range(start + 180, start + 360):
        wr.writerow([rows[l][1], rows[l][2], rows[l][3]])
    print(date[j][0] + " am saved.")
    
    for l in range(end - 360, end - 180):
        wr2.writerow([rows[l][1], rows[l][2], rows[l][3]])
    print(date[j][0] + " pm saved.")
        
w.close()
w2.close()
db.close()

2017-12-13 7:35:00 7:35:00
2017-12-13 17:35:00 17:35:00
2017-12-13 am saved.
2017-12-13 pm saved.
2017-12-27 7:42:00 7:42:00
2017-12-27 17:42:00 17:42:00
2017-12-27 am saved.
2017-12-27 pm saved.
2017-12-28 7:43:00 7:43:00
2017-12-28 17:43:00 17:43:00
2017-12-28 am saved.
2017-12-28 pm saved.
2018-01-03 7:44:00 7:44:00
2018-01-03 17:44:00 17:44:00
2018-01-03 am saved.
2018-01-03 pm saved.
2018-01-05 7:45:00 7:44:00
2018-01-05 17:44:00 17:44:00
2018-01-05 am saved.
2018-01-05 pm saved.
2018-01-06 7:44:00 7:44:00
2018-01-06 17:44:00 17:44:00
2018-01-06 am saved.
2018-01-06 pm saved.
2018-01-07 7:44:00 7:44:00
2018-01-07 17:44:00 17:44:00
2018-01-07 am saved.
2018-01-07 pm saved.
2018-02-16 7:19:00 7:19:00
2018-02-16 18:19:00 18:19:00
2018-02-16 am saved.
2018-02-16 pm saved.
2018-02-17 7:18:00 7:18:00
2018-02-17 18:18:00 18:18:00
2018-02-17 am saved.
2018-02-17 pm saved.
2018-02-24 7:09:00 7:09:00
2018-02-24 18:09:00 18:09:00
2018-02-24 am saved.
2018-02-24 pm saved.
2018-02-25 7:08:00 7

In [3]:
date = np.genfromtxt('../data/date_winter.csv', delimiter=',', dtype='str')
# date = np.genfromtxt('../data/test_date_summer.csv', delimiter=',', dtype='str')

import_data = np.loadtxt('../data/db_connect_data_am.csv', delimiter=',')
# import_data = np.loadtxt('../data/db_connect_test_data_am.csv', delimiter=',')
# import_data = np.loadtxt('../data/db_connect_data_pm.csv', delimiter=',')
# import_data = np.loadtxt('../data/db_connect_test_data_pm.csv', delimiter=',')

one = import_data[:,0] # swr
two = import_data[:,1] # mwr
thr = import_data[:,2] # lwr

swr = []
mwr = []
lwr = []

for cnt in range(len(date)): # 전체 데이터 날짜 수 (행)
    temp = []
    temp2 = []
    temp3 = []
    for i in range(180): # 한 날짜의 데이터들 한 행에 저장 (열)
        temp.append(one[i+180*cnt])
        temp2.append(two[i+180*cnt])
        temp3.append(thr[i+180*cnt])
    swr.append(temp)
    mwr.append(temp2)
    lwr.append(temp3)

w = open('../data/swr_ratio_calculation_am.csv', 'w', encoding='utf-8')
# w = open('../data/swr_ratio_test_calculation_am.csv', 'w', encoding='utf-8')
# w = open('../data/swr_ratio_calculation_pm.csv', 'w', encoding='utf-8')
# w = open('../data/swr_ratio_test_calculation_pm.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range(len(date)):
    all_hap = 0
    swr_hap = 0
    ratio = 0
    grade = 0
    
    for j in range(len(swr[i])):
        swr_hap += swr[i][j]
        all_hap += swr[i][j] + mwr[i][j] + lwr[i][j]
    
    ratio = round(swr_hap / all_hap * 100, 2)
    
#     if(ratio >= 18.00):
#         grade += 1
#     if(ratio >= 20.00):
#         grade += 1
#     if(ratio >= 22.00):
#         grade += 1
#     if(ratio >= 24.00):
#         grade += 1
        
    if(ratio >= 19.00):
        grade += 1
    if(ratio >= 21.00):
        grade += 1
    if(ratio >= 23.00):
        grade += 1
    if(ratio >= 24.00):
        grade += 1
    
    wr.writerow([ratio, grade])
w.close()

In [7]:
import_data = np.genfromtxt('../data/swr_ratio_calculation_am.csv', delimiter=',', dtype='float')
import_data2 = np.genfromtxt('../data/swr_ratio_test_calculation_am.csv', delimiter=',', dtype='float')
w = open('../data/swr_ratio_classification_input_am.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range(len(import_data)):
    wr.writerow(import_data[i][:])
for i in range(len(import_data2)):
    wr.writerow(import_data2[i][:])
w.close()

In [4]:
# 텐서플로우 모델 생성 위한 import

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
tf.set_random_seed(777)

# 초기값 선정 xavier 알고리즘
def xavier_init(n_inputs, n_outputs, uniform=True):
    
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

# 단파장 비율 학습 및 테스트 (편차 분포)

import_data = np.genfromtxt('../data/swr_ratio_calculation_am.csv', delimiter=',', dtype='float')

x_data = []
for i in range(len(import_data)):
    temp = []
    temp.append(import_data[i][0])
    x_data.append(temp)
    
# scaler = MinMaxScaler(feature_range=(0, 1))
# x_data = scaler.fit_transform(x_data)
test_x = x_data[9:]
x_data = x_data[:9]

y_data = []
test_y = []

for i in range(9):
    temp = []
    temp.append(import_data[i][1])
    y_data.append(temp)

raw_y = y_data
    
for i in range(9, len(import_data)):
    temp = []
    temp.append(import_data[i][1])
    test_y.append(temp)


y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

X = tf.placeholder("float", [None, 1])
Y = tf.placeholder("float", [None, 5])

nb_classes = 5

# W = tf.Variable(tf.random_normal([1, nb_classes]), name='weight')
# b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# Xavier Initializer 추가 코드
W = tf.get_variable("W", shape=[1, nb_classes], initializer=xavier_init(1, nb_classes))
b = tf.Variable(tf.zeros([nb_classes]))

H = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(H), axis=1))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 1000 == 0:
            train_accuracy = 0
            test_accuracy = 0
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))
            
            for i in range(len(x_data)):
                a = sess.run(H, feed_dict={X:[x_data[i]]})
                if(sess.run(tf.argmax(a, 1)) == (raw_y[i][0])):
                    train_accuracy += 1
            print("train accuracy = ", float(train_accuracy / len(x_data) * 100))

            for i in range(len(test_x)):
                a = sess.run(H, feed_dict={X:[test_x[i]]})
                if(sess.run(tf.argmax(a, 1)) == (test_y[i][0])):
                    test_accuracy += 1
                print(sess.run(tf.argmax(a, 1)))
            print("test accuracy = ", float(test_accuracy / len(test_x) * 100))

    print('-----------------------------')

    print('train_data = ', len(x_data), 'test_data = ', len(test_x))

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

0 25.920576
train accuracy =  11.11111111111111
[4]
[4]
[4]
test accuracy =  0.0
1000 1.401031
train accuracy =  55.55555555555556
[2]
[2]
[2]
test accuracy =  66.66666666666666
2000 1.3132244
train accuracy =  66.66666666666666
[2]
[1]
[2]
test accuracy =  100.0
3000 1.2097058
train accuracy =  66.66666666666666
[2]
[1]
[2]
test accuracy =  100.0
4000 1.106214
train accuracy =  66.66666666666666
[2]
[1]
[2]
test accuracy =  100.0
5000 1.0088959
train accuracy =  66.66666666666666
[2]
[1]
[2]
test accuracy =  100.0
6000 0.91913486
train accuracy =  66.66666666666666
[2]
[1]
[2]
test accuracy =  100.0
7000 0.83757263
train accuracy =  77.77777777777779
[2]
[1]
[2]
test accuracy =  100.0
8000 0.76469636
train accuracy =  77.77777777777779
[2]
[1]
[2]
test accuracy =  100.0
9000 0.70125836
train accuracy =  77.77777777777779
[2]
[1]
[2]
test accuracy =  100.0
10000 0.6464917
train accuracy =  77.77777777777779
[2]
[1]
[2]
test accuracy =  100.0
-----------------------------
train_data =  

In [ ]:
봄
[1] 일출 후 3시간 ~ 일출 후 6시간 정확도 64.28% (2로만 나옴) 37,14
[2] 일출 후 6시간 ~ 일몰 전 6시간-3시간 정확도 71.43% (3 두개, 나머지 모두 2) 37,14
[3] 일몰 전 6시간-3시간 ~ 일몰 전 3시간 정확도 64.28% (1 하나, 4 하나, 나머지 모두 0) 37,14 

여름
[1] 정확도 20% 20, 15
[2] 정확도 20% 20, 15
[3] 정확도 46.67% (1 두개, 나머지 모두 0) 20, 15

가을
[1] 정확도 0% 16, 8
[2] 정확도 50%
[3] 정확도 75% (모두 0으로 예측)

겨울
[1] 정확도 0% 9, 3
[2] 정확도 100%
[3] 정확도 33.33%